## Functions

__Function for extracting number from the string__

In [1]:
def num_extractor(string):
    
    import re
    
    # Regex expression for finding the number
    s = [float(s) for s in re.findall(r'-?\d+\.?\d*', string)]
    
    num = s
    
    if len(s) == 2:
    
        num = s[0]*1000 + s[1]
        
    if type(num) == list:
        
        num = float(num[0])
        
    return num

__Splitting the price in high and low__

In [2]:
def price_split(string):
    
    # Splitting algorithm
    split_string = string.split('-')
    len_split = len(split_string)
    
    
    low_price = num_extractor(split_string[0])
    high_price = num_extractor(split_string[0])
    
    # Vehicle has low and high price
    if len_split == 2:
        
        low_price = num_extractor(split_string[0])
        high_price = num_extractor(split_string[1])
        
    # Conversion of lakhs to numbers
    
    if low_price/10 < 1:
        
        low_price = low_price*1e5
        
    if high_price/10 < 1:
        
        high_price = high_price*1e5
    
        
    return (low_price,high_price)


__Specification Transposing Funstion__

In [3]:
def spec_transpose(df):
    
    # Column name
    spec_name = df['spec_columns'].values[0]
    
    # Renaming the specification value column
    df.rename(columns = {'spec_values':spec_name},inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df[['product_name',spec_name]]
    
    return df

## Data Import

In [4]:
import os

In [5]:
os.chdir('G:\Sanit_Horizontal\Data Analysis\web_scraping-main\Bikedekho\Data Files')

In [6]:
df = pd.read_csv('all_spec_extractor.csv', header = 0, encoding= 'unicode_escape')
df.drop(columns = [df.columns[0],df.columns[1]], axis = 1, inplace = True)

## Data Cleaning

In [7]:
spec_col = df.drop(columns = df.columns[-1], axis = 1)
spec_col.dropna(axis = 0,inplace = True)
spec_col.reset_index(inplace = True,drop = True)

In [8]:
spec_val = df.drop(columns = df.columns[-2], axis = 1)
spec_val.dropna(axis = 0,inplace = True)
spec_val.reset_index(inplace = True,drop = True)

In [9]:
all_spec = pd.concat([spec_col,spec_val['spec_values']],axis = 1)

In [10]:
all_spec.head()

,product_name,price,spec_columns,spec_values
0,Oben Rorr,Rs.1.03 Lakh*,Motor Power (w),1000
1,Oben Rorr,Rs.1.03 Lakh*,Starting,Push Button Start
2,Oben Rorr,Rs.1.03 Lakh*,Bluetooth Connectivity,"Bluetooth,WiFi"
3,Oben Rorr,Rs.1.03 Lakh*,Anti Theft Alarm,Yes
4,Oben Rorr,Rs.1.03 Lakh*,Speedometer,Digital


## Price Column Conversion

In [11]:
price_list = list(all_spec['price'])

In [12]:
low_list = []
high_list = []

In [13]:
for string in price_list:
    
#     print(string)
    low, high = price_split(string)
    
    low_list.append(low)
    high_list.append(high)   

In [14]:
all_spec = pd.concat([all_spec,pd.DataFrame({'price_low':low_list,'price_high':high_list})],axis = 1)

In [15]:
all_spec.head()

,product_name,price,spec_columns,spec_values,price_low,price_high
0,Oben Rorr,Rs.1.03 Lakh*,Motor Power (w),1000,103000.0,103000.0
1,Oben Rorr,Rs.1.03 Lakh*,Starting,Push Button Start,103000.0,103000.0
2,Oben Rorr,Rs.1.03 Lakh*,Bluetooth Connectivity,"Bluetooth,WiFi",103000.0,103000.0
3,Oben Rorr,Rs.1.03 Lakh*,Anti Theft Alarm,Yes,103000.0,103000.0
4,Oben Rorr,Rs.1.03 Lakh*,Speedometer,Digital,103000.0,103000.0


## Separting the brand list

In [16]:
brand = all_spec.drop(columns = ['spec_columns','spec_values'],axis = 1)
brand.drop_duplicates(inplace = True)
brand.reset_index(drop = True,inplace=True)

In [17]:
brand.head()

,product_name,price,price_low,price_high
0,Oben Rorr,Rs.1.03 Lakh*,103000.0,103000.0
1,Tork Kratos,Rs.1.22 - 1.37 Lakh*,122000.0,137000.0
2,Ampere Magnus EX,"Rs.81,900*",81900.0,81900.0
3,Bounce Infinity E1,"Rs.47,499 - 96,799*",47499.0,96799.0
4,Hero Electric Optima,"Rs.67,190 - 85,190*",67190.0,85190.0


In [18]:
spec_list = sorted(list(all_spec['spec_columns'].unique()))

## Spec to column conversion

__Transposing the all_spec DataFrame__

In [19]:
concat_df = brand.copy()

In [20]:
for spec in spec_list:
    
    dum = all_spec[all_spec['spec_columns']== spec] 
    concat_df = pd.merge(concat_df,spec_transpose(dum), how='outer')

C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_33360\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [21]:
concat_df.drop_duplicates(inplace=True)
concat_df.reset_index(drop=True,inplace=True)

In [22]:
list(concat_df.keys())

['product_name',
 'price',
 'price_low',
 'price_high',
 '0-100 Kmph (sec)',
 '0-40 Kmph (sec)',
 'ABS',
 'ARAI Mileage',
 'Acceleration (0-100 Kmph)',
 'Additional Features',
 'Additional Features Of Variant',
 'Additional Storage',
 'Adjustable Windscreen',
 'Anti Theft Alarm',
 'Artificial Exhaust Sound System',
 'Battery Capacity',
 'Battery Type',
 'Battery Warranty',
 'Bluetooth Connectivity',
 'Body Graphics',
 'Body Type',
 'Boot Space',
 'Brakes Front',
 'Brakes Rear',
 'Braking (60-0 Kmph)',
 'Braking Type',
 'Call/SMS Alerts',
 'Carry Hook',
 'Carry hook',
 'Central Locking',
 'Charger Output',
 'Charging At Charging Station',
 'Charging At Home',
 'Charging Network / Battery Swapping Network',
 'Charging Point',
 'Charging Station Locater',
 'Charging Time(0-100%)',
 'Charging Time(0-80%)',
 'Claimed Range',
 'Clock',
 'Continious Power',
 'Continuous Power',
 'Cooling System',
 'Cruise Control',
 'DRLs',
 'Display',
 'Distance to Empty Indicator',
 'Drive Type',
 'Dry Weig

## Extracting battery and motor related data

In [23]:
req_keys = []

In [24]:
for string in spec_list:
    
    if any([x in string for x in matches]):
        
        print(string)
        req_keys.append(string)

NameError: name 'matches' is not defined

In [ ]:
battery_motor_dataframe = concat_df[['product_name','price_low','price_high']+req_keys+['Drive Type']]

In [ ]:
battery_motor_dataframe.head()

## Swapable Battery Vehicles

In [ ]:
battery_motor_dataframe.keys()

In [ ]:
swappable_bat_vehicles = battery_motor_dataframe[~battery_motor_dataframe['Swappable Battery'].isnull()]
swappable_bat_vehicles.sort_values('price_high',inplace=True)

In [ ]:
swappable_bat_vehicles = swappable_bat_vehicles[['product_name','price_low','price_low','Battery Capacity','Battery Type','Motor Power (w)','Motor Type','Torque (Motor)','Drive Type']]
swappable_bat_vehicles.reset_index(drop = True, inplace=True)

In [ ]:
swappable_bat_vehicles

## Battery Capacity Conversion

In [25]:
concat_df.head()

,product_name,price,price_low,price_high,0-100 Kmph (sec),0-40 Kmph (sec),ABS,ARAI Mileage,Acceleration (0-100 Kmph),Additional Features,...,Tubeless Tyre,Turn Signal Lamp,Tyre Size,USB Charging Port,Underseat storage,Water Proof Rating,Wheel Size,Wheelbase,Wheels Type,Width
0,Oben Rorr,Rs.1.03 Lakh*,103000.0,103000.0,NaN,3s,NaN,NaN,NaN,"GPS, MHX (Maximum Heat Exchanging), Driver Ale...",...,Tubeless,LED,NaN,NaN,NaN,IP67,NaN,NaN,Alloy,NaN
1,Tork Kratos,Rs.1.22 - 1.37 Lakh*,122000.0,137000.0,NaN,3.5s,NaN,NaN,NaN,"Active Throttle Control, Front storage box, Ha...",...,Tubeless,LED,"Front :-90/80-17,\tRear :-120/80-17",Yes,NaN,NaN,"Front :-431.8 mm, Rear :-431.8 mm",1336 mm,Alloy,NaN
2,Ampere Magnus EX,"Rs.81,900*",81900.0,81900.0,NaN,10s,NaN,NaN,NaN,Front Glove Box,...,NaN,LED,NaN,NaN,Yes,NaN,"Front :-254 mm,Rear :-254 mm",1390 mm,Alloy,685 mm
3,Bounce Infinity E1,"Rs.47,499 - 96,799*",47499.0,96799.0,NaN,NaN,NaN,NaN,NaN,"EBS, Drag Mode, Drive Modes - Power | Eco, Loc...",...,Tubeless,LED,Front :-90/90-12\tRear :-120/70-12,NaN,12 L,IP67 (Battery),"Front :-304.8 mm,Rear :-304.8 mm",1260 mm,Alloy,695 mm
4,Hero Electric Optima,"Rs.67,190 - 85,190*",67190.0,85190.0,NaN,NaN,NaN,NaN,NaN,Walk Assist,...,NaN,NaN,"Front :- 90/90-12, \tRear :- 90/90-12",NaN,NaN,NaN,"Front :-304.8 mm,Rear :-304.8 mm",NaN,Alloy,NaN


In [27]:
battery_cap_list = list(concat_df['Battery Capacity'])

In [28]:
battery_cap_list

['4.4 kwh',
 '4 kWh',
 '60 V / 38.25 Ah',
 '1.9 KWh',
 '51.2 V / 30 Ah',
 '4 kWh',
 '5 kWh',
 '10.3 kWh',
 '4.8 kWh + 1.6 kWh',
 '50.4 V / 60.4 Ah',
 '3.24 KWh',
 '3.7 kWh',
 '4.56 kwh',
 '3 kWh',
 '3.94 kWh',
 '3.6 kWh',
 '48 V/20 Ah',
 '4.0 kWh',
 '48 V/12 Ah',
 '72 V/20 Ah',
 '3 kWh',
 '3 kWh',
 '1.25 KWH',
 '3.3 KWH',
 '48 V/56 Ah',
 '4.32 kWh',
 '60 V / 28 Ah',
 '3.8 Kwh',
 '60 V / 28 Ah',
 '2.0 kWh',
 '35 Ah',
 '72 V / 30 Ah',
 '35 Ah',
 '48V/20 Ah',
 '1.8 Kwh',
 nan,
 '60.8 V / 26 Ah',
 '48 V / 55 Ah',
 '4.4 kWh',
 nan,
 '60 V / 30 Ah',
 '48 V / 30 ah',
 '72 V / 28 Ah',
 '72 V / 30 Ah',
 '48 V / 40 Ah',
 '60 V / 30 Ah',
 '3.2 Kwh',
 '35 Ah',
 '2.3 kwh',
 '72 V / 31 Ah',
 '60 V, 20 Ah',
 '60 V / 35 Ah',
 '48 V / 30 Ah',
 '60 V/30 Ah',
 '3.2 kWh',
 '1.4 Kwh',
 '2.5 kWh',
 '1.56 Kwh',
 nan,
 '48 V/17.5 Ah',
 '60 V/26 Ah',
 '72 V, 30 Ah',
 '1.73 kwh',
 '72 V, 50 Ah',
 '48 V / 30 Ah',
 nan,
 '72 V / 23 Ah',
 '7.2 kwh',
 nan,
 '60 V/26 Ah',
 '72V/30 Ah',
 nan,
 '48 V/26 Ah',
 '20 Ah',

In [29]:
for string in battery_cap_list:
    
    if 'kwh' in string:
        
        print(string)

4.4 kwh
4.56 kwh


TypeError: argument of type 'float' is not iterable

## Export to csv

In [ ]:
# concat_df.to_csv('all_vehicle_spec_data.csv')